In [1]:
import torch
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="5"
import pandas as pd
import numpy as np
from tqdm import tqdm
sys.path.append("../")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from configs.listmle_gsage_xla_fused import Configs

In [3]:
CFG = Configs()

length of train: 66, length of valid: 14, length of test: 16


In [4]:
CFG.load_state_dict(os.path.join(CFG.OUTPUTDIR,"bestmodel_opa.pkl"),map_location="cpu")
model = CFG.model
model.cuda()
model.eval()
1

loading model checkpoint from epoch:  5247


1

In [5]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

total, trainable = count_parameters(model)
print(f"Total parameters: {total}")
print(f"Trainable parameters: {trainable}")


Total parameters: 1787135
Trainable parameters: 1787135


In [6]:
# CFG.test_dataset.files = CFG.test_dataset.files[:8]

In [7]:
USED_KEYS = ["node_features","node_config_features","node_separation","node_ops","edges","batches"]
pred_sequences = []
for info in tqdm(CFG.test_dataset):
    predictions = []
    for batch in CFG.stream_dataloder_collate([info]):
        with torch.no_grad():
            out = model(**{k:batch[k].cuda() for k in USED_KEYS}).cpu()
            predictions.append(out)
    pred_sequences.append(torch.concat(predictions).flatten()[:len(info["config_runtimes"])])

100%|██████████| 16/16 [03:51<00:00, 14.50s/it]


In [10]:
df = pd.DataFrame(CFG.test_dataset.files,columns=["ID"])
if CFG.test_dataset.is_tile:
    df["ID"] = df.ID.apply(lambda x: x.split("/")[-1].split(".")[0].replace("___",":").replace("test:",""))
    df["TopConfigs"] = [";".join([str(x) for x in ps.numpy().argsort().tolist()[:10]]) for ps in pred_sequences]
else:
    df["ID"] = df.ID.apply(lambda x: "layout:"+x.split("/")[-1].split(".")[0].replace("___",":").replace("test:",""))
    df["TopConfigs"] = [";".join([str(x) for x in ps.numpy().argsort().tolist()]) for ps in pred_sequences]

In [11]:
df.to_csv(os.path.join(CFG.OUTPUTDIR,"submission.csv"),index=False)

# Combine multiple files and save

In [12]:
import pandas as pd
import os

In [13]:
files = [
    "/app/nn-runtime-network/workdir/listmle_graphsage_tilemodel_updated/submission.csv",
    "/app/nn-runtime-network/workdir/listmle_graphsage_fused_xla_pairnorm/submission.csv",
    '../workdir/listmle_graphsage_nlp_fused_pairnorm/submission.csv'
]
make_zero = [
    False,False,False
]
# make_zero = [
#     True,True,True,True,True
# ]

In [14]:
pdfs =[]
for x,mask in zip(files,make_zero):
    tdf = pd.read_csv(x)
    if mask:
        tdf["TopConfigs"] = "0;1"
    pdfs.append(tdf)

In [15]:
tdf = pd.concat(pdfs)
tdf.to_csv(os.path.join("./submission","submission_fused_xla_fused_nlp_1115.csv"),index=False)

In [16]:
tdf

,ID,TopConfigs
0,tile:xla:0023795810403f8b0b244d88c901322f,1538;24;3720;4813;3616;555;3878;464;2536;1638
1,tile:xla:005c91ca7a50fffc663678fd44316f04,544;498;288;588;376;208;623;368;502;21
2,tile:xla:0070642211d5a98a16b94f4d7df229fe,618;518;763;741;292;188;189;805;26;135
3,tile:xla:008191e0c67a6e7a62cde1a3e1d66795,817;221;1171;195;176;292;1099;1077;726;792
4,tile:xla:008730b43f100be7c2800d7cb89578a4,241;206;473;336;729;281;668;766;316;280
...,...,...
29,layout:nlp:random:71b79ca6db513e7979c3702c5951...,998;996;993;994;992;997;999;995;418;96;280;506...
30,layout:nlp:random:7f6284ebe027b1e9a3850fc70385...,992;995;996;999;994;997;998;993;499;41;179;102...
31,layout:nlp:random:b2fdde3b72980907578648774101...,996;998;993;994;997;995;992;999;103;413;379;10...
32,layout:nlp:random:d15316c12eefdef1ba549eb43379...,994;997;998;999;992;995;993;996;140;511;717;13...
